In [1]:
!pip install huggingface_hub
!huggingface-cli login
!pip install wandb -qU
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.7 MB/s eta 0:00:00


In [2]:
!nvidia-smi | grep MiB

| N/A   35C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |


In [3]:
!pip install --quiet accelerate==0.23.0
!pip install --quiet peft==0.5.0
!pip install --quiet transformers
!pip install --quiet trl==0.4.7
!pip install --quiet wandb
!pip install --quiet evaluate
!pip install --quiet xformers
!pip install --quiet datasets
!pip install --quiet bitsandbytes
!pip install --quiet rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import time

In [5]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "llama-2-7b-sentiment-annotated3"

lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"
num_train_epochs = 2

fp16 = False
bf16 = False
per_device_train_batch_size = 24
per_device_eval_batch_size = 24
gradient_accumulation_steps = 1
gradient_checkpointing = False

max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

max_seq_length = 1024
packing = False
device_map = {"": 0}

https://www.reddit.com/r/LocalLLaMA/comments/15uu6lk/how_do_you_really_really_prompt_llama_2/
```
<s>[INST] <<SYS>>
{{system message}}
<</SYS>>
{{message}} [/INST] {{answer}} </s>
<s>[INST] {{message}} [/INST] {{answer}} </s>
<s>[INST] {{message}} [/INST]
```

In [6]:
# https://www.reddit.com/r/LocalLLaMA/comments/15uu6lk/how_do_you_really_really_prompt_llama_2/
def format_dolly(sample):
    instruction = f"""### You are solving the NLP problem. Annotate each word in the context as one of the
      following types: [<<PER>>, <<ORG>>, <<LOC>>, <<MISC>>].
      The entity type definitions are:
      PER is a person,
      ORG is an organization,
      LOC is a location, and
      MISC is an event, nationality, or product.
      The answer should only contain the annotated Context.
      """
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"{sample['response']}"
    prompt = "\n\n".join([i for i in ["[INST]", instruction, context, "### Answer\n", "[/INST]", response] if i is not None])
    return prompt

def format_template(sample):
    sample['text'] = f"{format_dolly(sample)}"
    return sample

def format_prediction(sample):
    answer = sample['response']
    sample['response'] = ''
    prompt = f"{format_template(sample)['text']}"
    return prompt

tag_to_ent_dic = {
    0: '0',
    1: 'PER',
    2: 'PER',
    3: 'ORG',
    4: 'ORG',
    5: 'LOC',
    6: 'LOC',
    7: 'MISC',
    8: 'MISC',
}

def ent_tag_to_response(ent_tag, token):
    if ent_tag != '0':
        return f"<<{ent_tag}>>"
    else:
        return token

def tags_to_response(sample):
    sample["ent_tags"] = [tag_to_ent_dic[ent_tag] for ent_tag in sample['ner_tags']]
    sample["context"] = " ".join(sample['tokens'])
    sample["response"] = [ent_tag_to_response(ent_tag, token) for ent_tag, token in zip(sample['ent_tags'], sample['tokens'])]
    sample["response"] = " ".join(sample["response"])
    return sample

In [7]:
from datasets import load_dataset

dataset = load_dataset("conll2003", split="train")
# dataset = dataset.select(range(200))
dataset = dataset.shuffle(seed=42)
dataset = dataset.map(tags_to_response)
dataset = dataset.map(format_template, remove_columns=dataset.column_names)
dataset = dataset.train_test_split(test_size=0.05, shuffle=False)

print(dataset['train'])
print(dataset['test'])
print(dataset['train'][0])
print(dataset['test'][0])

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 13338
})
Dataset({
    features: ['text'],
    num_rows: 703
})
{'text': '[INST]\n\n### You are solving the NLP problem. Annotate each word in the context as one of the\n      following types: [<<PER>>, <<ORG>>, <<LOC>>, <<MISC>>].\n      The entity type definitions are:\n      PER is a person,\n      ORG is an organization,\n      LOC is a location, and\n      MISC is an event, nationality, or product.\n      The answer should only contain the annotated Context.\n      \n\n### Context\n" Neither the National Socialists ( Nazis ) nor the communists dared to kidnap an American citizen , " he shouted , in an oblique reference to his extradition to Germany from Denmark . "\n\n### Answer\n\n\n[/INST]\n\n" Neither the <<MISC>> <<MISC>> ( <<MISC>> ) nor the communists dared to kidnap an <<MISC>> citizen , " he shouted , in an oblique reference to his extradition to <<LOC>> from <<LOC>> . "'}
{'text': '[INST]\n\n### You are solving the NLP probl

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    model_max_length=2048)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        bf16 = True
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map=device_map)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [10]:
!nvidia-smi | grep MiB

| N/A   36C    P0    48W / 400W |   5459MiB / 40960MiB |      0%      Default |


In [11]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb",
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    eval_steps=logging_steps*5,
    save_strategy="steps",
    metric_for_best_model='eval_loss',
    save_total_limit=3
)

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    "lm_head",
],
)


trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/13338 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [12]:
trainer.train()
trainer.model.save_pretrained(new_model)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
125,0.409600,0.323725
250,0.183100,0.306073
375,0.385800,0.294421
500,0.170800,0.288550
625,0.298900,0.284123
750,0.172900,0.282290
875,0.295000,0.280496
1000,0.173900,0.279754


In [13]:
# Free memory for merging weights
del model
torch.cuda.empty_cache()

In [14]:
!nvidia-smi | grep MiB

| N/A   47C    P0    53W / 400W |   8247MiB / 40960MiB |      0%      Default |


In [15]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model, device_map=device_map, torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = os.path.join(output_dir, "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [16]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:508: UserWarning: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration. This warning will be raised to an exception in v4.34.

Thrown during validation:
`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/foobar8675/llama-2-7b-sentiment-annotated3/commit/fe463a80df8333ff53b1d4ed6f0a8bad8f1edac5', commit_message='Upload tokenizer', commit_description='', oid='fe463a80df8333ff53b1d4ed6f0a8bad8f1edac5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from google.colab import runtime
runtime.unassign()